In [37]:
with open('emso.txt', 'r') as content_file:
    content = content_file.read()

doc = [x.strip() for x in ''.join(content).replace('\n', ' ').split('.') if x]
doc

['The year started with a partial unwind of the market euphoria of November and December',
 'Throughout January the US posted strong economic data',
 'While these new inputs added further credence to the view that the US is heading for a soft landing, they also pushed back against hopes that the US Federal Reserve (Fed) will need to ease monetary policy any time soon',
 'At the end of 2023, Fed Chairman Jerome Powell had guided that rate hikes were likely behind us and the time for cuts would soon be in sight, leading to an increase in pricing of cuts at the March FOMC meeting',
 'However, the January FOMC meeting proved more hawkish than expected with Chairman Powell indicating that a start to the cutting cycle in March was not their base case scenario',
 'Markets unwound easing expectations accordingly and the US dollar strengthened versus peers',
 'In emerging markets, sovereign bond issuance took off with a record start with USD 44',
 '3bn issued via primary markets in January',
 '

In [13]:
# from bertopic.representation import KeyBERTInspired
# from sentence_transformers import SentenceTransformer
# from bertopic import BERTopic
# 
# # Create a BERTopic model
# sentence_model = SentenceTransformer('ProsusAI/finbert')
# # We define a number of topics that we know are in the documents
# zeroshot_topic_list = ["credit", "performance", "market", "investment approach", "implementation", "stress test",
#                        "team", "organization", "fee", "redemption", "asia"]
# 
# # We fit our model using the zero-shot topics
# # and we define a minimum similarity. For each document,
# # if the similarity does not exceed that value, it will be used
# # for clustering instead.
# topic_model = BERTopic(
#     embedding_model=sentence_model,
#     min_topic_size=15,
#     zeroshot_topic_list=zeroshot_topic_list,
#     zeroshot_min_similarity=.85,
#     representation_model=KeyBERTInspired(),
# )
# topics, _ = topic_model.fit_transform(doc)
# topics_info = topic_model.get_topic_info()
# topics_info

No sentence-transformers model found with name ProsusAI/finbert. Creating a new one with MEAN pooling.


,Topic,Count,Name,Representation,Representative_Docs
0,-1,197,0_embi_gd_2025_2023,"[embi, gd, 2025, 2023, 2024, and, was, dollar,...",NaN
1,0,2,investment approach,"[financing, and, markets, carbon, , , , , , ]","[financing., CARBON FINANCING AND MARKETS]"
2,1,40,1_2022_pti_ppp_pml,"[2022, pti, ppp, pml, khans, nawaz, technocrat...",[reduced the risk of a protest movement spiral...


In [38]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')

model = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert')

inputs = tokenizer(doc, padding = True, truncation = True, return_tensors='pt') #tokenize text to be sent to model

outputs = model(**inputs)

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

positive = predictions[:, 0].tolist()

negative = predictions[:, 1].tolist()

neutral = predictions[:, 2].tolist()

table = {'Headline':doc, 'Positive':positive, 'Negative':negative, 'Neutral':neutral}

df2 = pd.DataFrame(table, columns = ['Headline', 'Positive', 'Negative', 'Neutral'])
df2

,Headline,Positive,Negative,Neutral
0,The year started with a partial unwind of the ...,0.134420,0.725172,0.140409
1,Throughout January the US posted strong econom...,0.950642,0.024710,0.024648
2,While these new inputs added further credence ...,0.846550,0.113676,0.039774
3,"At the end of 2023, Fed Chairman Jerome Powell...",0.893102,0.069036,0.037861
4,"However, the January FOMC meeting proved more ...",0.040008,0.931214,0.028778
...,...,...,...,...
143,Continued support for disaster resilience shou...,0.837201,0.048422,0.114377
144,This also goes hand-in-hand with continued sov...,0.080422,0.013122,0.906456
145,"Finally, we note that ESG benchmarks need to b...",0.107416,0.017741,0.874842
146,This could result in an unfair treatment of a ...,0.012026,0.944176,0.043799
